https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

In [1]:
!pip install transformers


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

  Using cached transformers-4.21.1-py3-none-any.whl (4.7 MB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)


In [2]:
%%capture
!pip install git+https://github.com/neuml/txtai datasets pandas

In [4]:
!conda install pytorch torchvision -c pytorch --yes


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pytorch
    - tor

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

ModuleNotFoundError: No module named 'torch'

In [42]:
df = pd.read_csv('/project/question-answers-processed/fin-ba-processed-combined.csv')

In [43]:
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df, test_size=0.2, random_state=42)

In [44]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1633 entries, 1849 to 1126
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1633 non-null   int64  
 1   question      1633 non-null   object 
 2   answers       1633 non-null   object 
 3   context_file  1633 non-null   object 
 4   answer_start  1633 non-null   float64
 5   answer_end    1633 non-null   float64
 6   context       1633 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 102.1+ KB


In [45]:
# Split df_dev into a validation and test set
df_dev, df_test = df_dev[:205], df_dev[205:]

In [46]:
# Training data
x=0
data_train=[]
while x < 1633:
  dataset = {"context": df_train['context'].iloc[x], "question": df_train['question'].iloc[x], "answers": df_train['answers'].iloc[x]}
           
  data_train.append(dataset)
  x+=1
#data

Baseline model 

In [23]:
from txtai.pipeline import HFTrainer
trainer = HFTrainer()

In [24]:
model_baseline, tokenizer_baseline = trainer("mbartolo/electra-large-synqa", data_train, task="question-answering", num_train_epochs=20)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.420700
1000,0.513600
1500,0.320700
2000,0.244100
2500,0.192300
3000,0.167300
3500,0.149800
4000,0.142600


In [25]:
from transformers import pipeline
questions_baseline = pipeline("question-answering", model=model_baseline, tokenizer=tokenizer_baseline)

In [26]:
df_dev.reset_index(drop=True, inplace=True)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
questions_baseline(df_dev.loc[1]['question'], df_dev.loc[1]['context'])['score']

0.0003400693240109831

In [29]:
result_baseline = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_dev.shape[0]):
    output = questions_baseline(df_dev['question'].values[i], df_dev['context'].values[i])
    output_df = pd.DataFrame({'question': df_dev['question'].values[i], 'answer': df_dev['answers'].values[i],'model_answer': output['answer'], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_baseline = pd.concat([result_baseline, output_df])

In [30]:
result_baseline['comparison'] = np.where(result_baseline['answer'] == result_baseline['model_answer'] , 'True', 'False')

In [31]:
result_baseline.to_csv('./result_baseline.csv')

In [32]:
result_baseline.comparison.value_counts()

False    124
True      81
Name: comparison, dtype: int64

In [33]:
import collections

# get tokens from text; just by splitting by spces
def get_simple_tokens(text):
    tokens = [token.strip() for token in text.split()]
    
    return tokens

# calculate f1 score for a single prediction-answer pair
def get_f1(pred, answer):
    pred_tokens = get_simple_tokens(pred)
    ans_tokens = get_simple_tokens(answer)
    
    common_tokens = collections.Counter(pred_tokens) & collections.Counter(ans_tokens)
    common_tokens_n = sum(common_tokens.values())
    
    if common_tokens_n == 0:
        return 0
    
    precision = 1.0 * common_tokens_n/len(pred_tokens)
    recall = 1.0 * common_tokens_n/len(ans_tokens)
    
    f1 = 2*((precision*recall)/(precision+recall))
    
    return f1

In [34]:
# loop through all answers, for df use for i in range(result_df.shape[0])

f1_scores = []
for i in range(result_baseline.shape[0]):
    f1_scores.append(get_f1(result_baseline['model_answer'].values[i], result_baseline['answer'].values[i]))
    
result_baseline['f1_score'] = f1_scores

In [35]:
result_baseline['f1_score'].mean()

0.47476446037078057

In [36]:
#model_baseline.save_pretrained('./bert_large_model_baseline')

In [24]:
#from transformers import AutoModel
#model = AutoModel.from_pretrained('./bert_large_model_baseline')

First Fine tune with Bert miniture google/bert_uncased_L-2_H-128_A-2

In [47]:
from datasets import load_dataset
from txtai.pipeline import HFTrainer

ds = load_dataset("squad_v2")

trainer_fine_tune1 = HFTrainer()
trainer_fine_tune1("google/bert_uncased_L-2_H-128_A-2", ds["train"].select(range(10000)), task="question-answering", output_dir="google/electra-large-discriminator")
print("Training complete")

2022-04-04 21:57:40,846 [WARNING] download_and_prepare: Reusing dataset squad_v2 (/home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-04-04 21:57:44,285 [WARNING] _map_single: Loading cached processed dataset at /home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-6659ecdda78b635a.arrow
Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuest

Step,Training Loss
500,4.333700
1000,3.668200
1500,3.366600
2000,3.269700
2500,3.140800
3000,3.024400
3500,2.981600


Training complete


In [48]:
model_finetune1, tokenizer_finetune1 = trainer_fine_tune1("google/electra-large-discriminator", df_train, task="question-answering", num_train_epochs=20)

Step,Training Loss
500,1.420700
1000,0.513600
1500,0.320700
2000,0.244100
2500,0.192300
3000,0.167300
3500,0.149800
4000,0.142600


In [49]:
from transformers import pipeline
questions_finetune1 = pipeline("question-answering", model=model_finetune1, tokenizer=tokenizer_finetune1)

In [50]:
result_finetune1 = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_dev.shape[0]):
    output = questions_finetune1(df_dev['question'].values[i], df_dev['context'].values[i])
    output_df = pd.DataFrame({'question': df_dev['question'].values[i], 'answer': df_dev['answers'].values[i],'model_answer': output['answer'], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_finetune1 = pd.concat([result_finetune1, output_df])

In [51]:
result_finetune1['comparison'] = np.where(result_finetune1['answer'] == result_finetune1['model_answer'] , 'True', 'False')

In [52]:
result_finetune1.head(25)

,question,answer,model_answer,confidence_score,start_index,end_index,comparison
0,Do I need basic finance knowledge for the Fina...,MSc Finance students are expected to have basi...,MSc,0.000334,0,3,False
1,"For MSc Finance, what is the structure of the ...",All participants study four core modules in Te...,.,0.000340,644,645,False
2,What is the requirement for GMAR or GRE exam f...,We do not require GMAT/GRE scores,We do not require GMAT/GRE scores,0.998889,378,411,True
3,"For the master program in Finance, who should ...",the Programme Administrator via email at: mgmt...,the,0.000716,76,79,False
4,What information do I need to provide for my M...,You will need to include a degree transcript i...,You,0.000285,0,3,False
5,"For the master program in Finance, what suppor...",please visit UCL's prospective students Corona...,pages,0.000326,388,393,False
6,Is ba master offerred as a part-time study?,"No, unfortunately this programme is not offere...","No, unfortunately this programme is not offere...",0.999099,0,69,True
7,"For MSc Finance, what are the contact details ...",the Programme Administrator via email at: mgmt...,the,0.001144,76,79,False
8,Can I apply for a part-time analytics master s...,"No, unfortunately this programme is not offere...","No, unfortunately this programme is not offere...",0.998493,0,69,True
9,Who should I contact for further details regar...,the Programme Administrator via email at: mgmt...,the,0.000709,76,79,False


In [53]:
result_finetune1.to_csv('./result_finetune1.csv')

In [54]:
result_finetune1.comparison.value_counts()

False    124
True      81
Name: comparison, dtype: int64

In [55]:
f1_scores = []
for i in range(result_finetune1.shape[0]):
    f1_scores.append(get_f1(result_finetune1['model_answer'].values[i], result_finetune1['answer'].values[i]))
    
result_finetune1['f1_score'] = f1_scores

In [56]:
result_finetune1['f1_score'].mean()

0.47476446037078057

In [ ]:
#model_finetune1.save_pretrained('Project/test/Esther/bert_large_model_finetune1')

Second Fine tune with google/bert_uncased_L-4_H-512_A-8

In [36]:
from datasets import load_dataset
#from txtai.pipeline import HFTrainer

ds = load_dataset("squad_v2")

trainer_finetune2 = HFTrainer()
trainer_finetune2("google/bert_uncased_L-4_H-512_A-8", ds["train"].select(range(10000)), task="question-answering", output_dir="bert-large-uncased-whole-word-masking-finetuned-squad")
print("Training complete")

2022-03-31 23:29:15,981 [WARNING] download_and_prepare: Reusing dataset squad_v2 (/home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-03-31 23:29:19,344 [WARNING] _map_single: Loading cached processed dataset at /home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-541f82dd7c186b64.arrow
Some weights of the model checkpoint at google/bert_uncased_L-4_H-512_A-8 were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuest

Step,Training Loss
500,2.778900
1000,2.083500
1500,1.545000
2000,1.297600
2500,1.215400
3000,0.859500
3500,0.855500


Training complete


In [37]:
model_finetune2, tokenizer_finetune2 = trainer_finetune2("bert-large-uncased-whole-word-masking-finetuned-squad", df_train, task="question-answering", num_train_epochs=20)

Step,Training Loss
500,0.275700
1000,0.046800
1500,0.021400
2000,0.008200
2500,0.002300
3000,0.001300
3500,0.000100
4000,0.000000


In [38]:
questions_finetune2 = pipeline("question-answering", model=model_finetune2, tokenizer=tokenizer_finetune2)

In [39]:
result_finetune2 = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_dev.shape[0]):
    output = questions_finetune2(df_dev['question'].values[i], df_dev['context'].values[i])
    output_df = pd.DataFrame({'question': df_dev['question'].values[i], 'answer': df_dev['answers'].values[i],'model_answer': output['answer'], 'context': df_dev['context'].values[i], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_finetune2 = pd.concat([result_finetune2, output_df])

In [40]:
result_finetune2['comparison'] = np.where(result_finetune2['answer'] == result_finetune2['model_answer'] , 'True', 'False')

In [41]:
result_finetune2.comparison.value_counts()

False    122
True      83
Name: comparison, dtype: int64

In [42]:
f1_scores = []
for i in range(result_finetune2.shape[0]):
    f1_scores.append(get_f1(result_finetune2['model_answer'].values[i], result_finetune2['answer'].values[i]))
    
result_finetune2['f1_score'] = f1_scores

In [43]:
result_finetune2['f1_score'].mean()

0.46584836340025

In [44]:
result_finetune2.to_csv('./result_finetune2.csv')

In [45]:
#model_finetune2.save_pretrained('Project/test/Esther/bert_large_model_finetune2')

Third fine tune with google/bert_uncased_L-8_H-512_A-8

In [59]:
from datasets import load_dataset
#from txtai.pipeline import HFTrainer

ds = load_dataset("squad_v2")

trainer_finetune3 = HFTrainer()
trainer_finetune3("google/bert_uncased_L-8_H-512_A-8", ds["train"].select(range(10000)), task="question-answering", output_dir="bert-large-uncased-whole-word-masking-finetuned-squad")
print("Training complete")

2022-04-01 01:34:39,484 [WARNING] download_and_prepare: Reusing dataset squad_v2 (/home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-04-01 01:34:42,840 [WARNING] _map_single: Loading cached processed dataset at /home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-6d2378291ddc6d3b.arrow
Some weights of the model checkpoint at google/bert_uncased_L-8_H-512_A-8 were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuest

Step,Training Loss
500,2.486400
1000,1.669000
1500,1.247700
2000,0.995400
2500,0.990600
3000,0.593900
3500,0.603300


Training complete


In [60]:
model_finetune3, tokenizer_finetune3 = trainer_finetune3("bert-large-uncased-whole-word-masking-finetuned-squad", df_train, task="question-answering", num_train_epochs=20)

Step,Training Loss
500,0.248800
1000,0.030700
1500,0.018700
2000,0.000500
2500,0.003400
3000,0.000500
3500,0.000100
4000,0.000000


In [61]:
from transformers import pipeline
questions_finetune3 = pipeline("question-answering", model=model_finetune3, tokenizer=tokenizer_finetune3)

In [62]:
result_finetune3 = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_dev.shape[0]):
    output = questions_finetune3(df_dev['question'].values[i], df_dev['context'].values[i])
    output_df = pd.DataFrame({'question': df_dev['question'].values[i], 'answer': df_dev['answers'].values[i],'model_answer': output['answer'], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_finetune3 = pd.concat([result_finetune3, output_df])

In [63]:
result_finetune3['comparison'] = np.where(result_finetune3['answer'] == result_finetune3['model_answer'] , 'True', 'False')

In [64]:
result_finetune3.comparison.value_counts()

False    122
True      83
Name: comparison, dtype: int64

In [65]:
result_finetune3

,question,answer,model_answer,confidence_score,start_index,end_index,comparison
0,Do I need basic finance knowledge for the Fina...,MSc Finance students are expected to have basi...,MSc,2.063777e-06,0,3,False
1,"For MSc Finance, what is the structure of the ...",All participants study four core modules in Te...,module,4.307857e-06,638,644,False
2,What is the requirement for GMAR or GRE exam f...,We do not require GMAT/GRE scores,We do not require GMAT/GRE scores,9.999952e-01,378,411,True
3,"For the master program in Finance, who should ...",the Programme Administrator via email at: mgmt...,the,4.052202e-06,76,79,False
4,What information do I need to provide for my M...,You will need to include a degree transcript i...,.,7.471557e-06,231,232,False
...,...,...,...,...,...,...,...
200,Is there a contact email for potential candita...,the Programme Administrator via: mgmt-ba@ucl.a...,the,4.495369e-06,76,79,False
201,Does my bachelor degree has to be highly numer...,we stress to our applicants that this programm...,.,7.933648e-07,226,227,False
202,What is the structure of the program for the m...,All participants study four core modules in Te...,All,4.302269e-06,0,3,False
203,"For the master program in Finance, what is the...","UK/EU/Overseas - £38,000","UK/EU/Overseas - £38,000",9.999933e-01,63,87,True


In [66]:
result_finetune3.to_csv('./result_finetune3.csv')

In [67]:
f1_scores = []
for i in range(result_finetune3.shape[0]):
    f1_scores.append(get_f1(result_finetune3['model_answer'].values[i], result_finetune3['answer'].values[i]))
    
result_finetune3['f1_score'] = f1_scores

In [68]:
result_finetune3['f1_score'].mean()

0.46628546162972856

In [69]:
model_finetune3.save_pretrained('Project/test/Esther/bert_large_model_finetune3')

In [70]:
#model_finetune3 = model_finetune3.from_pretrained('Project/test/Esther/bert_large_model_finetune3')

Forth fine tune with google/bert_uncased_L-12_H-768_A-12

In [71]:
from datasets import load_dataset
#from txtai.pipeline import HFTrainer

ds = load_dataset("squad_v2")

trainer_finetune4 = HFTrainer()
trainer_finetune4("google/bert_uncased_L-12_H-768_A-12", ds["train"].select(range(10000)), task="question-answering", output_dir="bert-large-uncased-whole-word-masking-finetuned-squad")
print("Training complete")

2022-04-01 04:07:40,048 [WARNING] download_and_prepare: Reusing dataset squad_v2 (/home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-04-01 04:07:43,440 [WARNING] _map_single: Loading cached processed dataset at /home/faculty/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-cad1646b6709c846.arrow
Some weights of the model checkpoint at google/bert_uncased_L-12_H-768_A-12 were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQue

Step,Training Loss
500,2.078500
1000,1.306500
1500,0.921700
2000,0.718500
2500,0.695900
3000,0.352900
3500,0.359200


Training complete


In [72]:
model_finetune3, tokenizer_finetune4 = trainer_finetune4("bert-large-uncased-whole-word-masking-finetuned-squad", df_train, task="question-answering", num_train_epochs=20)

Step,Training Loss
500,0.253300
1000,0.025100


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline
questions_finetune4 = pipeline("question-answering", model=model_finetune4, tokenizer=tokenizer_finetune4)

In [ ]:
result_finetune4 = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_dev.shape[0]):
    output = questions_finetune4(df_dev['question'].values[i], df_dev['context'].values[i])
    output_df = pd.DataFrame({'question': df_dev['question'].values[i], 'answer': df_dev['answers'].values[i],'model_answer': output['answer'], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_finetune4 = pd.concat([result_finetune4, output_df])

In [ ]:
result_finetune4['comparison'] = np.where(result_finetune4['answer'] == result_finetune4['model_answer'] , 'True', 'False')

In [ ]:
result_finetune4.comparison.value_counts()

In [ ]:
result_finetune4

In [ ]:
result_finetune4.to_csv('./result_finetune4.csv')

In [ ]:
f1_scores = []
for i in range(result_finetune4.shape[0]):
    f1_scores.append(get_f1(result_finetune4['model_answer'].values[i], result_finetune4['answer'].values[i]))

result_finetune4['f1_score'] = f1_scores

In [ ]:
result_finetune4['f1_score'].mean()

In [ ]:
model_finetune4.save_pretrained('Project/test/Esther/bert_large_model_finetune4')

Test

In [51]:
questions_test = pipeline("question-answering", model=model_finetune2, tokenizer=tokenizer_finetune2)

In [52]:
result_test = pd.DataFrame(columns=['question', 'answer', 'model_answer', 'confidence_score', 'start_index', 'end_index'])

for i in range(df_test.shape[0]):
    output = questions_test(df_test['question'].values[i], df_test['context'].values[i])
    output_df = pd.DataFrame({'question': df_test['question'].values[i], 'answer': df_test['answers'].values[i],'model_answer': output['answer'], 'confidence_score': output['score'], 'start_index':output['start'], 'end_index':output['end']}, index=[i])
    
    result_test = pd.concat([result_test, output_df])

In [53]:
result_test['comparison'] = np.where(result_test['answer'] == result_test['model_answer'] , 'True', 'False')

In [54]:
result_test.comparison.value_counts()

False    137
True      67
Name: comparison, dtype: int64

In [55]:
result_test.head(20)

,question,answer,model_answer,confidence_score,start_index,end_index,comparison
0,Do I have to be very quantitative for the mast...,we stress to our applicants that this programm...,.,1.145099e-06,226,227,False
1,"For MSc Finance, when is the deadline for appl...",31 March 2022,31 March 2022,9.999933e-01,160,173,True
2,"For MSc Finance, is experience in the financia...",Experience in the finance and finance-related ...,.,9.821463e-07,158,159,False
3,Can I talk to someone from the faculty to see ...,We consider such requests on a “case by case” ...,We consider such requests on a “case by case” ...,9.999971e-01,0,52,True
4,What are the programming language that I will ...,The core elements of the programme are deliver...,.,2.185180e-06,150,151,False
5,"For the master program in Finance, do I have t...",prior to 26 September 2022,prior to 26 September 2022,9.999981e-01,31,57,True
6,What career path do students of MSc Finance ha...,"commercial banks, investment banks, asset mana...",commercial,9.126628e-07,568,578,False
7,How many terms am I expected to study for Mast...,The majority of teaching finishes at the end o...,The,1.736272e-06,80,83,False
8,"For the master program in Finance, is there an...","Every year over 1,000 employers/organisations ...",Careers,2.858657e-06,76,83,False
9,What IELTS score is considered good for the ma...,an overall grade of 7.0 with a minimum of 6.5 ...,.,4.204385e-07,389,390,False


In [56]:
result_test.to_csv('./result_test.csv')

In [57]:
f1_scores = []
for i in range(result_test.shape[0]):
    f1_scores.append(get_f1(result_test['model_answer'].values[i], result_test['answer'].values[i]))
    
result_test['f1_score'] = f1_scores

In [58]:
result_test['f1_score'].mean()

0.3906535150059975